In [1]:

import pandas as pd
import numpy as np
import bs4 
import csv
import requests
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup


In [2]:
my_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
uClient = uReq(my_url) #opening connection grabbing page

In [4]:
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html,'html.parser') #html parsing


In [5]:
page_soup.h1 #testing

<h1 class="firstHeading" id="firstHeading" lang="en">List of postal codes of Canada: M</h1>

In [6]:
page_soup.td #testing

<td>M1A</td>

In [7]:
containers = page_soup.findAll("table",{"class": "wikitable sortable jquery-tablesorter"})
len(containers)

0

In [8]:
containers_2 = page_soup.findAll("td")
len(containers_2)
    
    

898

In [9]:
table = page_soup.find("table")

In [10]:
output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)
    


In [11]:

with open('output.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_rows)

In [12]:
df = pd.read_csv('output.csv', names = ['Postcode','Borough','Neighborhood'])
df

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


In [13]:
notAssigned = df[ df['Borough'] == 'Not assigned' ].index
df.drop(notAssigned, inplace=True)
df.head()
#remove not assigned Bouroughs

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211 entries, 2 to 286
Data columns (total 3 columns):
Postcode        211 non-null object
Borough         211 non-null object
Neighborhood    211 non-null object
dtypes: object(3)
memory usage: 6.6+ KB


In [15]:
df.Neighborhood = df.Neighborhood.str.replace('\n','')
df.head()
#remove /n                                            

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [16]:
df.Neighborhood = df.Neighborhood.str.replace('Not assigned','Queens Park')
df['Neighborhood'] = df[['Postcode','Borough','Neighborhood']].groupby(['Postcode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
clean_df = df[['Postcode','Borough','Neighborhood']].drop_duplicates()
clean_df
#group and concatenate Neighborhood fields, rename new dataframe

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Queens Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge,Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens,Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson,Garden District"


In [17]:
clean_df.shape
#newdataframe shape

(103, 3)

In [18]:
zip_csv = 'http://cocl.us/Geospatial_data'

zip_df = pd.read_csv(zip_csv)
#read csv

In [19]:
zip_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [20]:
df_merge = pd.merge(clean_df, zip_df, left_on='Postcode', right_on='Postal Code')
df_merge
#merge on different keys

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queens Park,M7A,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
7,M3B,North York,Don Mills North,M3B,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",M4B,43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",M5B,43.657162,-79.378937


In [21]:
df_merge = df_merge.drop(columns='Postal Code')
df_merge.head()
#remove dupe postal code

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queens Park,43.662301,-79.389494


In [22]:
df_merge

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queens Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
